In [33]:
import torch
!pip install d2l==1.0.0a0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [34]:
!pip install matplotlib
%matplotlib inline
!pip install matplotlib-inline
import sys
!{sys.executable} -m pip install matplotlib

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [35]:
import torch
import time
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import torch.nn as nn
import torch.nn.functional as F
from torchvision.datasets import CIFAR10
from torchvision.transforms import ToTensor
from torchvision.utils import make_grid
from torch.utils.data.dataloader import DataLoader
from torch.utils.data import random_split
import torch.optim as optim
%matplotlib inline
from torchvision import transforms
import torchvision.datasets as datasets
from torchvision import datasets, transforms
import torch.optim as optim
from d2l import torch as d2l
from torch.optim import lr_scheduler
import torch.optim as optim

In [36]:
data_dir = './data' 
transform = transforms.Compose([transforms.RandomHorizontalFlip(), transforms.ToTensor(),transforms.Resize(64)])
CIFER10_trainset = torchvision.datasets.CIFAR10(root=data_dir, train=True, download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(CIFER10_trainset, batch_size=10, shuffle=True)

Files already downloaded and verified


In [37]:
CIFER10_testset = torchvision.datasets.CIFAR10(root=data_dir, train=False, download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(CIFER10_testset, batch_size=10, shuffle=False)

Files already downloaded and verified


In [38]:
class Inception(nn.Module):
    # c1--c4 are the number of output channels for each branch
    def __init__(self, c1, c2, c3, c4, **kwargs):
        super(Inception, self).__init__(**kwargs)
        # Branch 1
        self.b1_1 = nn.LazyConv2d(c1, kernel_size=1)
        # Branch 2
        self.b2_1 = nn.LazyConv2d(c2[0], kernel_size=1)
        self.b2_2 = nn.LazyConv2d(c2[1], kernel_size=3, padding=1)
        # Branch 3
        self.b3_1 = nn.LazyConv2d(c3[0], kernel_size=1)
        self.b3_2 = nn.LazyConv2d(c3[1], kernel_size=5, padding=2)
        # Branch 4
        self.b4_1 = nn.MaxPool2d(kernel_size=3, stride=1, padding=1)
        self.b4_2 = nn.LazyConv2d(c4, kernel_size=1)

    def forward(self, x):
        b1 = F.relu(self.b1_1(x))
        b2 = F.relu(self.b2_2(F.relu(self.b2_1(x))))
        b3 = F.relu(self.b3_2(F.relu(self.b3_1(x))))
        b4 = F.relu(self.b4_2(self.b4_1(x)))
        return torch.cat((b1, b2, b3, b4), dim=1)

In [39]:
class GoogleNet(d2l.Classifier):
    def b1(self):
        return nn.Sequential(
            nn.LazyConv2d(64, kernel_size=7, stride=2, padding=3), 
            nn.ReLU(), nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [40]:
@d2l.add_to_class(GoogleNet)
def b2(self):
    return nn.Sequential(
        nn.LazyConv2d(64, kernel_size=1), nn.ReLU(),
        nn.LazyConv2d(192, kernel_size=3, padding=1),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [41]:
@d2l.add_to_class(GoogleNet)
def b3(self):
    return nn.Sequential(Inception(64, (96, 128), (16, 32), 32),
                         Inception(128, (128, 192), (32, 96), 64),
                         nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [42]:
@d2l.add_to_class(GoogleNet)
def b4(self):
    return nn.Sequential(Inception(192, (96, 208), (16, 48), 64),
                         Inception(160, (112, 224), (24, 64), 64),
                         Inception(128, (128, 256), (24, 64), 64),
                         Inception(112, (144, 288), (32, 64), 64),
                         Inception(256, (160, 320), (32, 128), 128),
                         nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

In [43]:
@d2l.add_to_class(GoogleNet)
def b5(self):
    return nn.Sequential(Inception(256, (160, 320), (32, 128), 128),
                         Inception(384, (192, 384), (48, 128), 128),
                         nn.AdaptiveAvgPool2d((1,1)), nn.Flatten())

In [44]:
@d2l.add_to_class(GoogleNet)
def __init__(self, lr=0.1, num_classes=10):
    super(GoogleNet, self).__init__()
    self.save_hyperparameters()
    self.net = nn.Sequential(self.b1(), self.b2(), self.b3(), self.b4(),
                             self.b5(), nn.LazyLinear(num_classes))
    self.net.apply(d2l.init_cnn)

In [45]:
GoogleNet()

GoogleNet(
  (net): Sequential(
    (0): Sequential(
      (0): LazyConv2d(0, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
      (1): ReLU()
      (2): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (1): Sequential(
      (0): LazyConv2d(0, 64, kernel_size=(1, 1), stride=(1, 1))
      (1): ReLU()
      (2): LazyConv2d(0, 192, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU()
      (4): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    )
    (2): Sequential(
      (0): Inception(
        (b1_1): LazyConv2d(0, 64, kernel_size=(1, 1), stride=(1, 1))
        (b2_1): LazyConv2d(0, 96, kernel_size=(1, 1), stride=(1, 1))
        (b2_2): LazyConv2d(0, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (b3_1): LazyConv2d(0, 16, kernel_size=(1, 1), stride=(1, 1))
        (b3_2): LazyConv2d(0, 32, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
        (b4_1): MaxPool2d(kernel_size=3, s

In [46]:
model = GoogleNet()
loss_fn = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [49]:
epoch_losses = [] 
model.train()
n_epochs=3
train_losses = np.zeros(n_epochs+1)
val_losses = np.zeros(n_epochs+1)
for epoch in range(n_epochs):
    running_loss = 0.0
    saved_loss = 0.0
    for i, data in enumerate(train_loader, 0):       
        inputs, labels = data
        inputs, labels = inputs, labels       
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, labels)
        
        loss.backward()
        optimizer.step()
      
        running_loss += loss.item()
        train_losses[epoch+1]=running_loss/1000
        if i % 1000 == 999:  
            print('%d, %5d| loss: %.3f' %(epoch+1, i+1, running_loss/1000))
            saved_loss = running_loss
            running_loss = 0.0
    epoch_losses.append(saved_loss/5000)

1,  1000| loss: 2.303
1,  2000| loss: 2.303
1,  3000| loss: 2.303
1,  4000| loss: 2.303
1,  5000| loss: 2.303
2,  1000| loss: 2.303
2,  2000| loss: 2.303
2,  3000| loss: 2.303
2,  4000| loss: 2.303
2,  5000| loss: 2.303
3,  1000| loss: 2.303
3,  2000| loss: 2.303
3,  3000| loss: 2.303
3,  4000| loss: 2.303
3,  5000| loss: 2.303


In [50]:
train_loader = torch.utils.data.DataLoader(CIFER10_trainset, batch_size=10, shuffle=True)
test_loader = torch.utils.data.DataLoader(CIFER10_testset, batch_size=10, shuffle=False)

def validate(model, train_loader, test_loader):
    for name, loader in [("train", train_loader), ("val", test_loader)]:
        correct = 0
        total = 0

        with torch.no_grad(): 
            for imgs, labels in loader:
                imgs=imgs
                outputs = model(imgs)
                _, predicted = torch.max(outputs, dim=1) 
                total += labels.shape[0]  
                correct += int((predicted == labels).sum()) 

        print("Accuracy {}: {:.2f}".format(name , correct / total))

validate(model, train_loader, test_loader)

Accuracy train: 0.10
Accuracy val: 0.10


In [51]:
pip install torchinfo

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [52]:
from torchinfo import summary

model = GoogleNet()
batch_size = 10
summary(model, input_size=(batch_size, 3, 64, 64))

Layer (type:depth-idx)                   Output Shape              Param #
GoogleNet                                [10, 10]                  --
├─Sequential: 1-1                        [10, 10]                  --
│    └─Sequential: 2-1                   [10, 64, 16, 16]          --
│    │    └─Conv2d: 3-1                  [10, 64, 32, 32]          9,472
│    │    └─ReLU: 3-2                    [10, 64, 32, 32]          --
│    │    └─MaxPool2d: 3-3               [10, 64, 16, 16]          --
│    └─Sequential: 2-2                   [10, 192, 8, 8]           --
│    │    └─Conv2d: 3-4                  [10, 64, 16, 16]          4,160
│    │    └─ReLU: 3-5                    [10, 64, 16, 16]          --
│    │    └─Conv2d: 3-6                  [10, 192, 16, 16]         110,784
│    │    └─ReLU: 3-7                    [10, 192, 16, 16]         --
│    │    └─MaxPool2d: 3-8               [10, 192, 8, 8]           --
│    └─Sequential: 2-3                   [10, 480, 4, 4]           --
│   